# PowerPoint Generator with Google Gemini


In [1]:
# Install required packages
!pip install google-generativeai python-pptx Pillow requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 16.6 MB/s eta 0:00:00


In [2]:
import os
import google.generativeai as genai
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import requests
from PIL import Image
import io
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

print("✅ All imports successful!")

✅ All imports successful!


In [9]:
class PPTGenerator:
    def __init__(self, api_key=None):
        """Initialize the PPT Generator with Gemini API"""
        self.api_key = api_key or os.getenv('GEMINI_API_KEY')
        if not self.api_key:
            raise ValueError("Gemini API key is required. Set GEMINI_API_KEY environment variable or pass it to the constructor.")

        # Configure Gemini
        genai.configure(api_key=self.api_key)
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        self.model_vision = genai.GenerativeModel('gemini-2.5-pro')

        # Initialize presentation
        self.presentation = Presentation()

    def _safe_string_conversion(self, data):
        """Safely convert any data type to string"""
        if isinstance(data, str):
            return data
        elif isinstance(data, list):
            # Join list items with newlines for content
            return '\n'.join(str(item) for item in data)
        elif data is None:
            return ""
        else:
            return str(data)

    def generate_content_outline(self, topic, num_slides=5):
        """Generate content outline using Gemini"""
        prompt = f"""
        Create a detailed outline for a PowerPoint presentation on "{topic}" with {num_slides} slides.
        Return the response as a JSON array with the following structure:
        [
            {{
                "title": "Slide Title",
                "content": "Main content points as bullet points",
                "slide_type": "title|content|image|conclusion"
            }}
        ]

        Make sure the content is engaging, informative, and well-structured.
        The response must be a valid JSON array.
        """

        try:
            response = self.model.generate_content(prompt)

            # Handle different response types
            content = None
            if hasattr(response, 'text') and response.text:
                content = response.text
            elif hasattr(response, 'candidates') and response.candidates:
                if hasattr(response.candidates[0].content, 'parts') and response.candidates[0].content.parts:
                    content = response.candidates[0].content.parts[0].text

            if content is None:
                print("No valid content received from Gemini API")
                return self._get_fallback_outline(topic, num_slides)

            # Ensure content is a string
            content = self._safe_string_conversion(content)
            content = content.strip()

            # Handle potential markdown code block
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0].strip()
            elif "```" in content:
                content = content.split("```")[1].strip()

            # Ensure we have a valid JSON string
            if not content.startswith('[') or not content.endswith(']'):
                print("Invalid JSON format received. Using fallback outline.")
                return self._get_fallback_outline(topic, num_slides)

            try:
                parsed_content = json.loads(content)
                # Validate that we got a list
                if not isinstance(parsed_content, list):
                    print("Expected JSON array, got different format. Using fallback outline.")
                    return self._get_fallback_outline(topic, num_slides)

                # Ensure all content fields are strings
                for slide in parsed_content:
                    if 'content' in slide:
                        slide['content'] = self._safe_string_conversion(slide['content'])
                    if 'title' in slide:
                        slide['title'] = self._safe_string_conversion(slide['title'])

                return parsed_content
            except json.JSONDecodeError as je:
                print(f"JSON parsing error: {je}")
                print(f"Raw content: {content[:200]}...")  # Show first 200 chars for debugging
                return self._get_fallback_outline(topic, num_slides)

        except Exception as e:
            print(f"Error generating content: {e}")
            return self._get_fallback_outline(topic, num_slides)

    def _get_fallback_outline(self, topic, num_slides):
        """Fallback outline if Gemini fails"""
        fallback_slides = [
            {
                "title": f"Introduction to {topic}",
                "content": "• Overview of the topic\n• Key objectives\n• What to expect",
                "slide_type": "title"
            },
            {
                "title": "Background Information",
                "content": "• Historical context\n• Current state\n• Important facts",
                "slide_type": "content"
            },
            {
                "title": "Key Concepts",
                "content": "• Main principles\n• Core ideas\n• Essential knowledge",
                "slide_type": "content"
            },
            {
                "title": "Applications and Examples",
                "content": "• Real-world applications\n• Case studies\n• Practical examples",
                "slide_type": "content"
            },
            {
                "title": "Benefits and Advantages",
                "content": "• Key benefits\n• Competitive advantages\n• Value proposition",
                "slide_type": "content"
            },
            {
                "title": "Challenges and Considerations",
                "content": "• Current challenges\n• Risk factors\n• Important considerations",
                "slide_type": "content"
            },
            {
                "title": "Future Outlook",
                "content": "• Emerging trends\n• Future developments\n• Growth opportunities",
                "slide_type": "content"
            },
            {
                "title": "Conclusion",
                "content": "• Summary of key points\n• Final thoughts\n• Next steps",
                "slide_type": "conclusion"
            }
        ]

        # Return only the requested number of slides
        return fallback_slides[:num_slides]

    def generate_image_description(self, slide_content):
        """Generate image description for slides using Gemini"""
        # Ensure slide_content is a string
        slide_content = self._safe_string_conversion(slide_content)

        prompt = f"""
        Based on this slide content, suggest a relevant image description that would enhance the presentation:

        {slide_content}

        Return only a brief, descriptive phrase suitable for image search (max 10 words).
        """

        try:
            response = self.model.generate_content(prompt)

            # Handle different response types
            result = None
            if hasattr(response, 'text') and response.text:
                result = response.text
            elif hasattr(response, 'candidates') and response.candidates:
                if hasattr(response.candidates[0].content, 'parts') and response.candidates[0].content.parts:
                    result = response.candidates[0].content.parts[0].text

            if result is None:
                return "professional business presentation"

            # Ensure result is a string
            result = self._safe_string_conversion(result)
            return result.strip()

        except Exception as e:
            print(f"Error generating image description: {e}")
            return "professional business presentation"

    def download_image(self, query, save_path="temp_image.jpg"):
        """Download a relevant image from Pexels based on the query"""
        try:
            # Ensure query is a string
            query = self._safe_string_conversion(query)

            # Pexels API endpoint
            url = "https://api.pexels.com/v1/search"

            # Headers with API key
            try:
                from google.colab import userdata
                pexels_api = userdata.get('PEXELS_API_KEY')
            except:
                # Fallback if not in Colab
                pexels_api = os.getenv('PEXELS_API_KEY')

            if not pexels_api:
                print("Pexels API key not found, creating placeholder image")
                raise ValueError("Pexels API key not found")

            headers = {
                'Authorization': pexels_api
            }

            # Search parameters
            params = {
                'query': query,
                'per_page': 1,
                'orientation': 'landscape'
            }

            # Get search results
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()

            # Parse response
            data = response.json()
            if not data.get('photos'):
                raise ValueError(f"No images found for query: {query}")

            # Get the first photo's original size URL
            image_url = data['photos'][0]['src']['original']

            # Download the image
            img_response = requests.get(image_url)
            img_response.raise_for_status()

            # Save the image
            with open(save_path, 'wb') as f:
                f.write(img_response.content)

            return save_path

        except Exception as e:
            print(f"Error downloading image: {e}")
            # Fallback to placeholder if there's an error
            try:
                from PIL import Image
                img = Image.new('RGB', (800, 600), color='#4A90E2')
                img.save(save_path)
                return save_path
            except Exception as e:
                print(f"Error creating placeholder image: {e}")
                return None

    def create_title_slide(self, title, subtitle=""):
        """Create a title slide"""
        # Ensure inputs are strings
        title = self._safe_string_conversion(title)
        subtitle = self._safe_string_conversion(subtitle)

        slide_layout = self.presentation.slide_layouts[0]  # Title slide layout
        slide = self.presentation.slides.add_slide(slide_layout)

        # Set title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)
        title_shape.text_frame.paragraphs[0].font.bold = True
        title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        # Set subtitle if provided
        if subtitle:
            subtitle_shape = slide.placeholders[1]
            subtitle_shape.text = subtitle
            subtitle_shape.text_frame.paragraphs[0].font.size = Pt(24)
            subtitle_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        return slide

    def create_content_slide(self, title, content, include_image=False):
        """Create a content slide with bullet points"""
        # Ensure inputs are strings
        title = self._safe_string_conversion(title)
        content = self._safe_string_conversion(content)

        slide_layout = self.presentation.slide_layouts[1]  # Title and content layout
        slide = self.presentation.slides.add_slide(slide_layout)

        # Set title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(36)
        title_shape.text_frame.paragraphs[0].font.bold = True

        # Set content
        content_shape = slide.placeholders[1]
        content_shape.text = content

        # Format bullet points
        text_frame = content_shape.text_frame
        for paragraph in text_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        # Add image if requested
        if include_image:
            try:
                print(f"Generating image description for content: {content[:50]}...")
                image_desc = self.generate_image_description(content)
                print(f"Generated image description: {image_desc}")

                image_path = self.download_image(image_desc)
                if image_path and os.path.exists(image_path):
                    slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
                    # Clean up temporary image
                    os.remove(image_path)
            except Exception as e:
                print(f"Error adding image: {e}")

        return slide

    def create_image_slide(self, title, content, image_query):
        """Create a slide with image and text"""
        # Ensure inputs are strings
        title = self._safe_string_conversion(title)
        content = self._safe_string_conversion(content)
        image_query = self._safe_string_conversion(image_query)

        slide_layout = self.presentation.slide_layouts[8]  # Blank layout
        slide = self.presentation.slides.add_slide(slide_layout)

        # Add title
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
        title_frame = title_box.text_frame
        title_frame.text = title
        title_frame.paragraphs[0].font.size = Pt(36)
        title_frame.paragraphs[0].font.bold = True
        title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        # Add content
        content_box = slide.shapes.add_textbox(Inches(0.5), Inches(2), Inches(4), Inches(5))
        content_frame = content_box.text_frame
        content_frame.text = content

        # Format content
        for paragraph in content_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        # Add image
        try:
            image_path = self.download_image(image_query)
            if image_path and os.path.exists(image_path):
                slide.shapes.add_picture(image_path, Inches(5), Inches(2), height=Inches(5))
                os.remove(image_path)
        except Exception as e:
            print(f"Error adding image: {e}")

        return slide

    def generate_presentation(self, topic, num_slides=5, output_path="generated_presentation.pptx"):
        """Generate a complete PowerPoint presentation"""
        print(f"Generating presentation on: {topic}")

        # Generate content outline
        outline = self.generate_content_outline(topic, num_slides)

        if not outline:
            print("Failed to generate outline. Using basic template.")
            outline = self._get_fallback_outline(topic, num_slides)

        # Create slides based on outline
        for i, slide_data in enumerate(outline):
            title = slide_data.get("title", f"Slide {i+1}")
            content = slide_data.get("content", "Content placeholder")
            slide_type = slide_data.get("slide_type", "content")

            # Ensure all data is properly converted to strings
            title = self._safe_string_conversion(title)
            content = self._safe_string_conversion(content)
            slide_type = self._safe_string_conversion(slide_type)

            print(f"Creating slide {i+1}: {title}")

            try:
                if i == 0 or slide_type == "title":
                    self.create_title_slide(title, f"Generated by Gemini AI")
                elif slide_type == "image":
                    image_query = self.generate_image_description(content)
                    self.create_image_slide(title, content, image_query)
                else:
                    include_image = (i % 3 == 0 and i > 0)  # Add image every 3rd slide, but not the first
                    self.create_content_slide(title, content, include_image)

                print(f"✅ Successfully created slide {i+1}")

            except Exception as e:
                print(f"❌ Error creating slide {i+1}: {e}")
                # Create a basic content slide as fallback
                try:
                    self.create_content_slide(title, content, False)
                    print(f"✅ Created fallback slide {i+1}")
                except Exception as e2:
                    print(f"❌ Even fallback slide creation failed: {e2}")

        # Save presentation
        try:
            self.presentation.save(output_path)
            print(f"✅ Presentation saved as: {output_path}")
            return output_path
        except Exception as e:
            print(f"❌ Error saving presentation: {e}")
            return None

print("✅ Completely fixed PPTGenerator class defined successfully!")

✅ Completely fixed PPTGenerator class defined successfully!


## Setup Gemini API Key

You need to set up your Gemini API key. You can either:
1. Create a `.env` file with `GEMINI_API_KEY=your_api_key_here`
2. Set it directly in the code below

Get your API key from: https://makersuite.google.com/app/apikey

In [10]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
#api_key = ""
if api_key:
    print("✅ Gemini API key is configured")
else:
    print("❌ Please set your GEMINI_API_KEY in the .env file or uncomment the line above")

✅ Gemini API key is configured


## Generate a Presentation

Now let's create a presentation! You can change the topic and number of slides as needed.

In [11]:
# Initialize the generator

try:
    generator = PPTGenerator(api_key=api_key)
    print("✅ PPT Generator initialized successfully!")
except ValueError as e:
    print(f"❌ Error: {e}")
    print("Please set your GEMINI_API_KEY first.")

✅ PPT Generator initialized successfully!


In [12]:
# Generate a presentation
topic = "Artificial Intelligence in Business"  # Change this to your desired topic
num_slides = 5  # Change this to your desired number of slides

try:
    output_file = generator.generate_presentation(topic, num_slides, "ai_business_presentation.pptx")
except Exception as e:
    print(e)

Generating presentation on: Artificial Intelligence in Business
Creating slide 1: Artificial Intelligence: The New Engine of Business Growth
✅ Successfully created slide 1
Creating slide 2: What is AI and Why is it a Game-Changer?
✅ Successfully created slide 2
Creating slide 3: AI in Action: Transforming Core Business Functions
✅ Successfully created slide 3
Creating slide 4: Implementing AI: A Strategic Roadmap
Generating image description for content: Step 1: Identify Business Problems, Not Solutions....
Generated image description: A strategic roadmap for digital business transformation.
✅ Successfully created slide 4
Creating slide 5: The Future is Intelligent: Seizing the AI Opportunity
✅ Successfully created slide 5
✅ Presentation saved as: ai_business_presentation.pptx
